In [ ]:
import pandas as pd
import os

In [ ]:
class company_prerocessing():
#     def __init__()
    # 파일 열기
    def open_df(folder_path):
        df = pd.read_csv(folder_path)
        return df

    # 재무 상태표
    def bs_preprocessing(df_bs, cmp_list):

        if df_bs.columns[2].isdigit() == True:
            try:
                df_bs_1 = df_bs.iloc[:, [0, 2, 3, 4]]
            except:
                pass

        elif df_bs.columns[6].isdigit() == True:
            df_bs_1 = df_bs.iloc[:, [1, 6, 7, 8]]

        elif df_bs.columns[7].isdigit() == True:
            df_bs_1 = df_bs.iloc[:, [1, 7, 8, 9]]

        elif df_bs.columns[8].isdigit() == True:
            df_bs_1 = df_bs.iloc[:, [1, 8, 9, 10]]

        else:
            df_bs_1 = df_bs.iloc[:, 0:4]

        df_bs_1.columns = ['', df_bs_1.columns[1], df_bs_1.columns[2], df_bs_1.columns[3]]
        df_bs_1 = df_bs_1.T

        df_bs_1 = df_bs_1.rename(columns=df_bs_1.iloc[0])
        df_bs_1 = df_bs_1.drop(df_bs_1.index[0])
        df_bs_1.drop('label_ko', inplace=True, axis =1)

        df_bs_1 = df_bs_1.reset_index().rename(columns={'index': 'date'})


        df_bs_1['company'] = cmp_list
        print('bs_preprocessing complete')

        return df_bs_1

    # 포괄손익 계산서
    def cis_preprocessing(df_cis, cmp_list):

        if '01-20' in df_cis.columns[6]:
            if '01-20' in df_cis.columns[8]:
                df_cis_1 = df_cis.iloc[:, [1, 6, 7, 8]]
            else:
                df_cis_1 = df_cis.iloc[:, [1, 6, 7]]

        else:
            if '01-20' in df_cis.columns[9]:
                df_cis_1 = df_cis.iloc[:, [1, 7, 8, 9]]
            else:
                df_cis_1 = df_cis.iloc[:, [1, 7, 8]]

        if len(df_cis_1.columns) == 4:
            df_cis_1.columns = ['', df_cis_1.columns[1], df_cis_1.columns[2], df_cis_1.columns[3]]
        elif len(df_cis_1.columns) == 3:
            df_cis_1.columns = ['', df_cis_1.columns[1], df_cis_1.columns[2]]

        df_cis_1 = df_cis_1.T

        df_cis_1 = df_cis_1.rename(columns=df_cis_1.iloc[0])
        df_cis_1 = df_cis_1.drop(df_cis_1.index[0])
        df_cis_1.drop('label_ko', inplace=True, axis =1)

        df_cis_1 = df_cis_1.reset_index().rename(columns={'index': 'date'})

        df_cis_1['company'] = cmp_list
        df_cis_1['date'] = df_cis_1['date'].str[9:]
        print('cis_preprocessing complete')

        return df_cis_1

    # 데이터 프레임 저장
    def save_df(folder_path, df, cmp_list):
        df.to_csv(folder_path + cmp_list + '.csv', index=False)


In [ ]:
# if '__main__' == __name__:

FOLDER_PATH = 'data/최종/'

# 원본 PATH
PATH1 = '재무상태표/'
PATH2 = '포괄손익계산서/'

# 전처리 PATH
PATH3 = '재무재표 전처리/'
PATH4 = '포괄손익계산서 전처리/'

# 파일 리스트화
file_list1 = os.listdir(FOLDER_PATH +'재무상태표/')
file_list2 = os.listdir(FOLDER_PATH +'포괄손익계산서/')

var_1_dic = {'자본총계': 4, '자산총계': 4, '매입채무': 13, '매출채권': 13, '부채총계': 4, '유동부채': 4, '유동자산': 4, '재고자산': 4}
var_2_dic = {'매출총이익': 5, '법인세비용차감': 15, '당기순이익': 10}

bs_a = pd.DataFrame(columns=['date', 'company'])
cis_a = pd.DataFrame(columns=['date', 'company'])

for cmp_list1, cmp_list2 in zip(file_list1, file_list2):
    # 재무 상태표
    df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + PATH1  + cmp_list1))
    df_bs_1 = company_prerocessing.bs_preprocessing(df, cmp_list1[:-13])
    company_prerocessing.save_df(os.path.join(FOLDER_PATH + PATH3), df_bs_1, cmp_list1[:-13])

    # 변수 전처리
    list1 = ['date', 'company']
    for var1, var1_ in var_1_dic.items():
        for bs in df_bs_1.columns:
            if var1 in bs:
                if var1_ >= len(bs):
                    list1.append(bs)
    print(df_bs_1[list1])
    bs_a = pd.concat([bs_a, df_bs_1[list1]])

    # 포괄 손익 계산서
    df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + PATH2 + cmp_list2))
    df_cis_1 = company_prerocessing.cis_preprocessing(df, cmp_list2[:-15])
    company_prerocessing.save_df(os.path.join(FOLDER_PATH + PATH4), df_cis_1, cmp_list2[:-15])

    # 변수 전처리
    list2 = ['date', 'company']
    for var2, var2_ in var_2_dic.items():
        for cis in df_cis_1.columns:
            if var2 in cis:
                if var2_ >= len(cis):
                    list1.append(cis)
    print(df_cis_1[list2])
    cis_a = pd.concat([cis_a, df_cis_1[list2]])

bs_a.reset_index(inplace=True, drop=True)
cis_a.reset_index(inplace=True, drop=True)
company_prerocessing.save_df(os.path.join(FOLDER_PATH + PATH3), bs_a, '재무상태표 통합')
company_prerocessing.save_df(os.path.join(FOLDER_PATH + PATH4), cis_a, '포괄손익계산서 통합')


In [ ]:
var_1_dic = {'자본총계': 4, '자산총계': 4, '매입채무': 13, '매출채권': 13, '부채총계': 4, '유동부채': 4, '유동자산': 4, '재고자산': 4}
var_2_dic = {'매출총이익': 5, '법인세비용차감': 15, '당기순이익': 10}

In [ ]:
df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + '재무상태표/' + file_list1[0]))
df_bs_1 = company_prerocessing.bs_preprocessing(df, cmp_list1[:-13])
company_prerocessing.save_df(os.path.join(FOLDER_PATH + '재무재표 전처리/'), df_bs_1, cmp_list1[:-13])

In [ ]:
list1 = ['date', 'company']
for var1, var1_ in var_1_dic.items():
    for bs in df_bs_1.columns:
        if var1 in bs:
            if var1_ == len(bs):
                print(bs)
                list1.append(bs)
# print(df_bs_1[list1])
# bs_a = pd.concat([bs_a, df_bs_1[list1]])

In [ ]:
len('매출채권 및 기타유동채권')